# Module 6: Update your app

### Objectives

* Perform a rolling update using kubectl.

### Updating an application

Users expect applications to be available all the time and developers are expected to deploy new versions of them several times a day. In Kubernetes this is done with rolling updates. **Rolling updates** allow Deployments' update to take place with zero downtime by incrementally updating Pods instances with new ones. The new Pods will be scheduled on Nodes with available resources.

In the previous module we scaled our application to run multiple instances. This is a requirement for performing updates without affecting application availability. By default, the maximum number of Pods that can be unavailable during the update and the maximum number of new Pods that can be created, is one. Both options can be configured to either numbers or percentages (of Pods). In Kubernetes, updates are versioned and any Deployment update can be reverted to a previous (stable) version.

> Rolling updates allow Deployments' update to take place with zero downtime by incrementally updating Pods instances with new ones.

## Rolling updates overview

<div>
<img src="https://d33wubrfki0l68.cloudfront.net/30f75140a581110443397192d70a4cdb37df7bfc/fa906/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates1.svg" width=400>

<img src="https://d33wubrfki0l68.cloudfront.net/678bcc3281bfcc588e87c73ffdc73c7a8380aca9/703a2/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates2.svg" width=400>
</div>

<div>
<img src="https://d33wubrfki0l68.cloudfront.net/9b57c000ea41aca21842da9e1d596cf22f1b9561/91786/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates3.svg" width=400>

<img src="https://d33wubrfki0l68.cloudfront.net/6d8bc1ebb4dc67051242bc828d3ae849dbeedb93/fbfa8/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates4.svg" width=400>
 
</div>
    


Similar to application Scaling, if a Deployment is exposed publicly, the Service will load-balance the traffic only to available Pods during the update. An available Pod is an instance that is available to the users of the application.

Rolling updates allow the following actions:

* Promote an application from one environment to another (via container image updates)
* Rollback to previous versions
* Continuous Integration and Continuous Delivery of applications with zero downtime

> If a Deployment is exposed publicly, the Service will load-balance the traffic only to available Pods during the update.



In the following lab we'll update our application to a new version, and also perform a rollback.

### Lab 6 - Update your app
#### Step 1 of 3: Update the version of the app

To list your deployments use the get deployments command: `kubectl get deployments`


In [1]:
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
POD_NAME=$(kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}' | grep $studentId | head -n 1) 
port=$(expr $(echo ${studentId} | tr -cd '[[:digit:]]') + 8000)

kubectl get deployments 


NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp-student1051   4/4     4            4           35m
kubernetes-bootcamp-student1052   1/1     1            1           25m
kubernetes-bootcamp-student1054   4/4     4            4           35m
kubernetes-bootcamp-student1056   1/1     1            1           23m
kubernetes-bootcamp-student1058   3/4     2            3           35m
kubernetes-bootcamp-student1059   1/1     1            1           33m
kubernetes-bootcamp-student1063   4/4     4            4           35m
kubernetes-bootcamp-student1065   1/1     1            1           24m
kubernetes-bootcamp-student1066   4/4     4            4           34m
kubernetes-bootcamp-student1070   4/4     4            4           36m
kubernetes-bootcamp-student1073   4/4     4            4           33m
kubernetes-bootcamp-student1074   1/1     1            1           25m
kubernetes-bootcamp-student1075   4/4     4            4           35m
kubern

To list the running Pods use the `get pods` command:

In [2]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

NAME                                               READY   STATUS    RESTARTS   AGE
kubernetes-bootcamp-student1086-859899d75d-5kgk4   1/1     Running   0          8m29s
kubernetes-bootcamp-student1086-859899d75d-j8kjh   1/1     Running   0          8m29s
kubernetes-bootcamp-student1086-859899d75d-wg7mz   1/1     Running   0          35m
kubernetes-bootcamp-student1086-859899d75d-z2bth   1/1     Running   0          8m29s


To view the current image version of the app, run a `describe` command against the Pods (look at the Image field):

In [3]:
kubectl describe deployments/kubernetes-bootcamp-$studentId

Name:                   kubernetes-bootcamp-student1086
Namespace:              k8shacktenant
CreationTimestamp:      Wed, 28 Apr 2021 15:23:47 +0000
Labels:                 app=kubernetes-bootcamp-student1086
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=kubernetes-bootcamp-student1086
Replicas:               4 desired | 4 updated | 4 total | 4 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=kubernetes-bootcamp-student1086
  Containers:
   kubernetes-bootcamp:
    Image:        gcr.io/google-samples/kubernetes-bootcamp:v1
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Progressing    True    NewReplicaSetAvailable
  Available      True    MinimumReplicasAvailable

To update the image of the application to version 2, use the `set image` command, followed by the deployment name and the new image version:

In [4]:
kubectl set image deployments/kubernetes-bootcamp-$studentId kubernetes-bootcamp=jocatalin/kubernetes-bootcamp:v2

deployment.apps/kubernetes-bootcamp-student1086 image updated


The command notified the Deployment to use a different image for your app and initiated a rolling update. Check the status of the new Pods, and view the old one terminating with the `get pods` command:

> To execute a multiple times, use CTRL-Enter while positioned on that cell.

In [5]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

NAME                                               READY   STATUS              RESTARTS   AGE
kubernetes-bootcamp-student1086-6cddcb6f9b-4zrfk   0/1     ContainerCreating   0          6s
kubernetes-bootcamp-student1086-6cddcb6f9b-8hcxz   1/1     Running             0          6s
kubernetes-bootcamp-student1086-6cddcb6f9b-msnht   0/1     ContainerCreating   0          0s
kubernetes-bootcamp-student1086-859899d75d-5kgk4   1/1     Terminating         0          8m39s
kubernetes-bootcamp-student1086-859899d75d-j8kjh   1/1     Running             0          8m39s
kubernetes-bootcamp-student1086-859899d75d-wg7mz   1/1     Running             0          35m
kubernetes-bootcamp-student1086-859899d75d-z2bth   1/1     Terminating         0          8m39s


#### Step 2 of 3: Verify an update

First, let’s check that the App is running. To find out the exposed IP and Port we can use `describe service`:

In [6]:
kubectl describe deployments/kubernetes-bootcamp-$studentId

Name:                   kubernetes-bootcamp-student1086
Namespace:              k8shacktenant
CreationTimestamp:      Wed, 28 Apr 2021 15:23:47 +0000
Labels:                 app=kubernetes-bootcamp-student1086
Annotations:            deployment.kubernetes.io/revision: 2
Selector:               app=kubernetes-bootcamp-student1086
Replicas:               4 desired | 3 updated | 5 total | 3 available | 2 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=kubernetes-bootcamp-student1086
  Containers:
   kubernetes-bootcamp:
    Image:        jocatalin/kubernetes-bootcamp:v2
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    ReplicaSetUpdated
OldReplicaSets: 

Create an environment variable called NODE_PORT that has the value of the Node port assigned.
And another one that compute NODE_NAME from a  `kubectl describe pods ` command. 

> Before we do so, let's recreate the service we deleted at the end of previous module.

In [7]:
kubectl expose deployment/kubernetes-bootcamp-$studentId --type="NodePort" --port 8080

service/kubernetes-bootcamp-student1086 exposed


In [8]:
NODE_PORT=$(kubectl get services/kubernetes-bootcamp-$studentId -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=$NODE_PORT
NODE_NAME=$(kubectl describe pods $POD_NAME | grep Node: | head -n 1)
NODE_NAME=${NODE_NAME#Node:} 
NODE_NAME=${NODE_NAME%/*} 
NODE_NAME=$(echo $NODE_NAME | xargs)
echo NODE_NAME=$NODE_NAME


NODE_PORT=30615
NODE_NAME=worker27.etc.fr.comm.hpecorp.net


Next, we’ll do a `curl` to the the exposed IP and port:

In [9]:
curl http://$NODE_NAME:$NODE_PORT

Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-student1086-6cddcb6f9b-nrlhc | v=2


We hit a different Pod with every request and we see that all Pods are running the latest version (v2).

The update can be confirmed also by running a rollout status command:

In [10]:
kubectl rollout status deployments/kubernetes-bootcamp-$studentId

deployment "kubernetes-bootcamp-student1086" successfully rolled out


To view the current image version of the app, run a describe command against the Pods:

In [11]:
kubectl describe pods -l app=kubernetes-bootcamp-$studentId

Name:         kubernetes-bootcamp-student1086-6cddcb6f9b-4zrfk
Namespace:    k8shacktenant
Priority:     0
Node:         worker28.etc.fr.comm.hpecorp.net/16.31.84.70
Start Time:   Wed, 28 Apr 2021 15:56:20 +0000
Labels:       app=kubernetes-bootcamp-student1086
              pod-template-hash=6cddcb6f9b
Annotations:  cni.projectcalico.org/podIP: 10.192.1.119/32
              cni.projectcalico.org/podIPs: 10.192.1.119/32
              kubernetes.io/psp: hcp-psp-privileged
Status:       Running
IP:           10.192.1.119
IPs:
  IP:           10.192.1.119
Controlled By:  ReplicaSet/kubernetes-bootcamp-student1086-6cddcb6f9b
Containers:
  kubernetes-bootcamp:
    Container ID:   docker://f86ba2eb52e41596b6939d72520c3d43d49cb447c1083f64920fa2464ed9af1b
    Image:          jocatalin/kubernetes-bootcamp:v2
    Image ID:       docker-pullable://jocatalin/kubernetes-bootcamp@sha256:fb1a3ced00cecfc1f83f18ab5cd14199e30adc1b49aa4244f5d65ad3f5feb2a5
    Port:           <none>
    Host Port:      <n

We run now version 2 of the app (look at the Image field)

#### Step 3 of 3: Rollback an update

Let’s perform another update, and deploy image tagged as `v10` :

In [13]:
kubectl set image deployments/kubernetes-bootcamp-$studentId kubernetes-bootcamp=gcr.io/google-samples/kubernetes-bootcamp:v10

Use `get deployments` to see the status of the deployment:

In [14]:
kubectl get deployment

NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp-student1051   3/4     2            3           38m
kubernetes-bootcamp-student1052   1/1     1            1           27m
kubernetes-bootcamp-student1054   4/4     4            4           37m
kubernetes-bootcamp-student1056   1/1     1            1           25m
kubernetes-bootcamp-student1058   3/4     2            3           38m
kubernetes-bootcamp-student1059   1/1     1            1           36m
kubernetes-bootcamp-student1063   4/4     4            4           37m
kubernetes-bootcamp-student1065   1/1     1            1           26m
kubernetes-bootcamp-student1066   4/4     4            4           36m
kubernetes-bootcamp-student1070   4/4     4            4           38m
kubernetes-bootcamp-student1073   4/4     4            4           36m
kubernetes-bootcamp-student1074   1/1     1            1           27m
kubernetes-bootcamp-student1075   4/4     4            4           37m
kubern

And something is wrong… We do not have the desired number of Pods available. List the Pods again:

In [15]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

NAME                                               READY   STATUS             RESTARTS   AGE
kubernetes-bootcamp-student1086-6cddcb6f9b-4zrfk   1/1     Running            0          2m21s
kubernetes-bootcamp-student1086-6cddcb6f9b-8hcxz   1/1     Running            0          2m21s
kubernetes-bootcamp-student1086-6cddcb6f9b-nrlhc   1/1     Running            0          2m12s
kubernetes-bootcamp-student1086-78d8b87cb6-bpgws   0/1     ImagePullBackOff   0          53s
kubernetes-bootcamp-student1086-78d8b87cb6-grkmb   0/1     ErrImagePull       0          53s


A `describe` command on the Pods should give more insights:

In [16]:
kubectl describe pods -l app=kubernetes-bootcamp-$studentId

Name:         kubernetes-bootcamp-student1086-6cddcb6f9b-4zrfk
Namespace:    k8shacktenant
Priority:     0
Node:         worker28.etc.fr.comm.hpecorp.net/16.31.84.70
Start Time:   Wed, 28 Apr 2021 15:56:20 +0000
Labels:       app=kubernetes-bootcamp-student1086
              pod-template-hash=6cddcb6f9b
Annotations:  cni.projectcalico.org/podIP: 10.192.1.119/32
              cni.projectcalico.org/podIPs: 10.192.1.119/32
              kubernetes.io/psp: hcp-psp-privileged
Status:       Running
IP:           10.192.1.119
IPs:
  IP:           10.192.1.119
Controlled By:  ReplicaSet/kubernetes-bootcamp-student1086-6cddcb6f9b
Containers:
  kubernetes-bootcamp:
    Container ID:   docker://f86ba2eb52e41596b6939d72520c3d43d49cb447c1083f64920fa2464ed9af1b
    Image:          jocatalin/kubernetes-bootcamp:v2
    Image ID:       docker-pullable://jocatalin/kubernetes-bootcamp@sha256:fb1a3ced00cecfc1f83f18ab5cd14199e30adc1b49aa4244f5d65ad3f5feb2a5
    Port:           <none>
    Host Port:      <n

There is no image called `v10` in the repository. Let’s roll back to our previously working version. We’ll use the `rollout` undo command:

In [17]:
kubectl rollout undo deployments/kubernetes-bootcamp-$studentId

deployment.apps/kubernetes-bootcamp-student1086 rolled back


The `rollout` command reverted the deployment to the previous known state (v2 of the image). Updates are versioned and you can revert to any previously know state of a Deployment. List again the Pods:

> To execute a multiple times, use CTRL-Enter while positioned on that cell.

In [18]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

NAME                                               READY   STATUS    RESTARTS   AGE
kubernetes-bootcamp-student1086-6cddcb6f9b-4zrfk   1/1     Running   0          3m5s
kubernetes-bootcamp-student1086-6cddcb6f9b-6b7nl   1/1     Running   0          26s
kubernetes-bootcamp-student1086-6cddcb6f9b-8hcxz   1/1     Running   0          3m5s
kubernetes-bootcamp-student1086-6cddcb6f9b-nrlhc   1/1     Running   0          2m56s


Four Pods are running. Check again the image deployed on the them:

In [19]:
kubectl describe pods -l app=kubernetes-bootcamp-$studentId

Name:         kubernetes-bootcamp-student1086-6cddcb6f9b-4zrfk
Namespace:    k8shacktenant
Priority:     0
Node:         worker28.etc.fr.comm.hpecorp.net/16.31.84.70
Start Time:   Wed, 28 Apr 2021 15:56:20 +0000
Labels:       app=kubernetes-bootcamp-student1086
              pod-template-hash=6cddcb6f9b
Annotations:  cni.projectcalico.org/podIP: 10.192.1.119/32
              cni.projectcalico.org/podIPs: 10.192.1.119/32
              kubernetes.io/psp: hcp-psp-privileged
Status:       Running
IP:           10.192.1.119
IPs:
  IP:           10.192.1.119
Controlled By:  ReplicaSet/kubernetes-bootcamp-student1086-6cddcb6f9b
Containers:
  kubernetes-bootcamp:
    Container ID:   docker://f86ba2eb52e41596b6939d72520c3d43d49cb447c1083f64920fa2464ed9af1b
    Image:          jocatalin/kubernetes-bootcamp:v2
    Image ID:       docker-pullable://jocatalin/kubernetes-bootcamp@sha256:fb1a3ced00cecfc1f83f18ab5cd14199e30adc1b49aa4244f5d65ad3f5feb2a5
    Port:           <none>
    Host Port:      <n

We see that the deployment is using a stable version of the app (v2). The Rollback was successful.

This terminates module 6 and the workshop, let's perform some cleanup.

In [20]:
kubectl delete services/kubernetes-bootcamp-$studentId
kubectl delete deployment/kubernetes-bootcamp-$studentId


service "kubernetes-bootcamp-student1086" deleted
deployment.apps "kubernetes-bootcamp-student1086" deleted


[Return to main Notebook](./1-WKSHP-K8S101.ipynb).